In [ ]:
!sudo apt-get -y update
!sudo apt-get -y install python3-pip
!sudo apt-get -y install python-is-python3
!wget https://github.com/PINTO0309/onnx2tf/releases/download/1.16.31/flatc.tar.gz \
  && tar -zxvf flatc.tar.gz \
  && sudo chmod +x flatc \
  && sudo mv flatc /usr/bin/
!pip install -U pip \
  && pip install tensorflow==2.19.0 \
  && pip install ai_edge_litert==1.2.0 \
  && pip install -U onnx==1.17.0 \
  && python -m pip install onnx_graphsurgeon \
        --index-url https://pypi.ngc.nvidia.com \
  && pip install -U onnxruntime==1.18.1 \
  && pip install -U onnxsim==0.4.33 \
  && pip install -U simple_onnx_processing_tools \
  && pip install -U onnx2tf \
  && pip install -U protobuf==3.19.6 \
  && pip install -U h5py==3.11.0 \
  && pip install -U psutil==5.9.5 \
  && pip install -U ml_dtypes==0.5.1 \
  && pip install -U tf-keras==2.19.0 \
  && pip install flatbuffers>=23.5.26

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,926 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:10 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,245 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Package

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -U --no-deps \
tensorflowjs \
tensorflow_decision_forests \
ydf \
tensorflow_hub

!onnx2tf -i /content/drive/MyDrive/age_gender_new_new.onnx -ois input:1,3,224,224 -osd -dgc

!tensorflowjs_converter \
--input_format tf_saved_model \
--output_format tfjs_graph_model \
saved_model \
tfjs_model!

E0000 00:00:1746885362.896844    2782 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746885362.904128    2782 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1746885362.930590    2782 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1746885362.930641    2782 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1746885362.930646    2782 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1746885362.930655    2782 computation_placer.cc:177] computation placer already registered. Please check linka

In [ ]:
from google.colab import drive
drive.mount('/content/drive')          # run once per session – follow the auth link

# pick a folder name inside Drive (creates it if missing)
DEST = "/content/drive/MyDrive/new_model"

!mkdir -p "$DEST"
!cp -r /content/saved_model  "$DEST/"
!cp -r /content/tfjs_model!   "$DEST/"

print("✅ copied to", DEST)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ copied to /content/drive/MyDrive/new_model


In [ ]:


import tensorflow as tf
import numpy as np
from PIL import Image

# 1. Load the SavedModel you pointed tensorflowjs_converter at
model = tf.saved_model.load("/content/saved_model")
infer = model.signatures["serving_default"]

# 2. Prepare your image
def preprocess(path):
    img = Image.open(path).convert("RGB").resize((224,224))
    arr = np.array(img).astype(np.float32) / 255.0
    # If you used ImageNet normalization:
    mean = np.array([0.485, 0.456, 0.406], dtype=np.float32)
    std  = np.array([0.229, 0.224, 0.225], dtype=np.float32)
    arr = (arr - mean) / std
    return tf.expand_dims(arr, axis=0)  # shape: (1,224,224,3)

# path to your test image
img_path = "/content/drive/MyDrive/308909_v9_bb.jpg"
input_tensor = preprocess(img_path)

# 3. Run inference
outputs = infer(input_tensor)

# 4. Print raw outputs
# Depending on your model signature, keys might be something like 'age_head' and 'gender_head'
for name, tensor in outputs.items():
    print(f"{name}: {tensor.numpy().squeeze()}")

# e.g. you might see:
#   age_logits: 32.847
#   gender_head/MatMul: 0.7235   (this is before sigmoid if you didn’t bake it in)


AttributeError: module 'numpy' has no attribute 'dtypes'